In [1]:
import pandas as pd
from gensim.parsing.preprocessing import preprocess_string
from gensim import corpora
import gensim
from gensim.models.coherencemodel import CoherenceModel
import re
# from tabulate import tabulate

from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import numpy as np
import os 
import numpy as np
from nltk.stem.snowball import SnowballStemmer
import pickle

# import image module
from IPython.display import Image

In [5]:
# # List stop words
import nltk
from nltk.corpus import stopwords
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('omw-1.4')
# nltk.download('averaged_perceptron_tagger')
# # print(stopwords.words("english"))

In [6]:
import random
# Set the global random seed
random.seed(42)
np.random.seed(42)

### Helper functions

In [26]:
##########################################################################################################################
# Get the filtered data for each country
def get_country_filtered_data(df, country_name):
    df['region_of_origin'] = df.region_of_origin.apply(lambda x: str(x).strip())
    print('####################################')
    print('Statistics for the whole dataset')
    print('####################################')
    print(f'Total number of articles in the dataset (for all the countries): \033[1;32m{df.shape[0]:,}\033[0m')
    print(f'Total number of articles missing in the dataset (for all the countries): \033[1;31m{df.text.isna().sum():,}\033[0m')
    print(f'Percentage of missing articles: \033[1;31m{round((df.text.isna().sum()/df.shape[0])*100, 2):,}%\033[0m')
    print('\n\n')
    countries_dict = {'AUSTR': 'AUS', 'USA': 'USA', 'UK': 'UK', 'INDIA': 'IND', 'CAN': 'CAN'}
    df['country'] = df.region_of_origin.apply(lambda x: countries_dict[country_name] if re.search(country_name, x) else 'other')
    df_country = df[df.country == countries_dict[country_name]]
    df_country = df_country[['text', 'region_of_origin', 'publication_date', 'publisher_name']]
    df_country.text.drop_duplicates(inplace=True)

    print('####################################')
    print(f'Statistics for \033[1m{countries_dict[country_name]}\033[0m dataset')
    print('####################################')

    print(f'    Total number of articles retrieved for \033[1;32m{country_name}\033[0m, without any duplicates: \033[1;32m{df_country.shape[0]:,}\033[0m')
    print(f'    Total number of columns retrieved for \033[1;32m{country_name}\033[0m: \033[1;32m{df_country.shape[1]}\033[0m')
    print('-------------------------------------------------------------------------------')

    # Treating the missing articles in 'text' column
    missing_rows = df_country.text.isna().sum()
    print(f'    Total number of missing articles for \033[1;31m{country_name}\033[0m: \033[1;31m{missing_rows:,}\033[0m')
    print(f'    Percentage of missing articles for \033[1;31m{country_name}\033[0m: \033[1;31m{round((missing_rows/df_country.shape[0])*100, 2):,}%\033[0m')
    if missing_rows > 0:
        df_country.dropna(inplace=True)
        print('-------------------------------------------------------------------------------')
        print(f'    Total number of articles after treating the missing articles: \033[1;32m{df_country.shape[0]:,}\033[0m')
    return df_country
##########################################################################################################################


# Preprocessing functions
##########################################################################################################################
# Keeping longer words
def words_length(x):
    if (len(x) > 3 and x.isalpha()):
        return x
    else:
        return '....removeThispart....'
##########################################################################################################################


##########################################################################################################################
stopwords_re = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')

# Removal
def preprocess_data(df):
    df['preprocessed_sentence'] = df.text.apply(lambda x: re.sub(r'(?i)\bhttps?://[^\s/$.?#].[^\s]*\b', '', x))
    print('Done removing web links...')
    df['preprocessed_sentence'] = df.preprocessed_sentence.map(lambda x: re.sub(r"[^a-zA-Z0-9\n\t]|\d+", " ", str(x)).lower())
    print('Done removing numbers, single letter words, new lines, tab spaces...')
    df['preprocessed_sentence'] = df.preprocessed_sentence.apply(lambda x: stopwords_re.sub('', x))
    print('Done removing the stop words...')
    df['preprocessed_sentence'] = df.preprocessed_sentence.apply(lambda x: re.sub(r" +", " ",x))
    print('Done removing extra spaces after removing the links, words, numbers...')
    df['preprocessed_sentence'] = df.preprocessed_sentence.apply(lambda x: np.vectorize(words_length)(np.array(x.split(' '))) )
    print('Done selecting specific word lengths...')
    df['preprocessed_sentence'] = df.preprocessed_sentence.apply(lambda x: re.sub(r"....removeThispart....", " ", " ".join(x)))
    print('Clean up done...')
    df['preprocessed_sentence'] = df.preprocessed_sentence.apply(lambda x: re.sub(r" +", " ", str(x)).strip().split(' '))
    print('Final clean up done...')

    # Output progress
    print(f"\nPreprocessing completed for {len(df)} articles!")

    return df
##########################################################################################################################



##########################################################################################################################
# Snowball stemming
#the stemmer requires a language parameter
snow_stemmer = SnowballStemmer(language='english')

def snowball_stemm(x):
    stemm = snow_stemmer.stem(x)

    return stemm
##########################################################################################################################


##########################################################################################################################
# Save the dictionary and bag of words
def save_dict_bow(dictionary, bow_corpus, country_name):
    dictionary.save(f'cc-bigrams-trigrams-{country_name}-australia.dict')

    with open(f'bow_corpus_bigrams-trigrams_{country_name}_australia.pickle', 'wb') as f:
        pickle.dump(bow_corpus, f)


# load the dictionary and bag of words locally
def load_dict_bow(country_name):
    dictionary = gensim.corpora.Dictionary.load(f'../../../australia/cc-bigrams-trigrams-{country_name}-australia.dict')

    with open(f'../../../australia/bow_corpus_bigrams-trigrams_{country_name}_australia.pickle', 'rb') as f:
        bow = pickle.load(f)
    return dictionary, bow
##########################################################################################################################


##########################################################################################################################
# Output writer
import csv

def output_writer(df, step_name):
# Open a CSV file for writing with UTF-8 encoding
    with open(f'{step_name}_output_australia.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f, quoting=csv.QUOTE_NONE, escapechar='\\', delimiter='|')
        # Write the header row to the CSV file
        writer.writerow(df.columns)

        # Write the dataframe to the CSV file
        writer.writerows(df.values)
        # Write the dataframe to the CSV file
        # writer.writerows(df.values)
##########################################################################################################################

### Reading the data

In [10]:
# Read the data into a dataframe
df = pd.read_csv('../../../../financial-wellbeing-all-in-one.csv')

In [11]:
df.columns

Index(['article_title', 'text', 'relevant', 'copyright', 'subject_codes',
       'art', 'modification_datetime', 'company_codes_occur_ticker_exchange',
       'company_codes_occur', 'company_codes_about', 'company_codes_lineage',
       'company_codes_ticker_exchange',
       'company_codes_relevance_ticker_exchange', 'publication_date',
       'market_index_codes', 'credit', 'section',
       'company_codes_association_ticker_exchange', 'currency_codes',
       'company_codes_about_ticker_exchange', 'region_of_origin',
       'company_codes_lineage_ticker_exchange', 'ingestion_datetime',
       'modification_date', 'source_name', 'language_code', 'region_codes',
       'company_codes_association', 'person_codes', 'byline',
       'company_codes_relevance', 'source_code', 'an', 'word_count',
       'company_codes', 'industry_codes', 'title', 'publication_datetime',
       'publisher_name', 'action', 'document_type', 'dateline',
       'relevancy_score'],
      dtype='object')

In [12]:
# Get the filtered data for Australia
# Use 'AUSTR' for Australia
# Use 'UK' for UK
# Use 'INDIA' for India
# Use 'USA' for USA
# Use 'CAND' for Canada

df_aus = get_country_filtered_data(df, "AUSTR")

####################################
Statistics for the whole dataset
####################################
Total number of articles in the dataset (for all the countries): 1,510,798
Total number of articles missing in the dataset (for all the countries): 0
Percentage of missing articles: 0.0%



####################################
Statistics for AUS dataset
####################################
    Total number of articles retrieved for AUSTR, without any duplicates: 119,947
    Total number of columns retrieved for AUSTR: 4
-------------------------------------------------------------------------------
    Total number of missing articles for AUSTR: 0
    Percentage of missing articles for AUSTR: 0.0%


In [13]:
df_aus

,text,region_of_origin,publication_date,publisher_name
3790,"Forming a company key for one family HOW, wh...",AUSNZ AUSTR,1120089600000,West Australian Newspapers Limited
4390,"There are many different types of homebuyers, ...",AUSNZ AUSTR,1493424000000,Fairfax Media Management Pty Limited
4391,Off-the-plan contracts review The NSW Governme...,AUSNZ AUSTR,1517616000000,Fairfax Media Management Pty Limited
4392,'Super must rise to 12%': MP Whitlam MP Stephe...,AUSNZ AUSTR,1570579200000,Fairfax Media Management Pty Limited
4393,Where there's a will TODAY I'd like to talk ab...,AUSNZ AUSTR,1558742400000,Fairfax Media Management Pty Limited
...,...,...,...,...
1508690,Super choice brings costs ONE in two small b...,AUSNZ AUSTR,1130889600000,Nationwide News Pty Ltd.
1508691,"Home loans guide NO doc, low doc, combinatio...",AUSNZ AUSTR,1155686400000,Nationwide News Pty Ltd.
1508692,Taxing times for families TAXPAYERS having the...,AUSNZ AUSTR,1216771200000,Nationwide News Pty. Ltd.
1508697,Legal papers vital Legal papers vital W...,AUSNZ AUSTR,1188950400000,Nationwide News Pty Ltd.


In [14]:
# Delete the bigger dataset to save on memory
del df

### Preprocessing the data

In [15]:
# Preprocess the data
df_aus = preprocess_data(df_aus)

Done removing web links...
Done removing numbers, single letter words, new lines, tab spaces...
Done removing the stop words...
Done removing extra spaces after removing the links, words, numbers...
Done selecting specific word lengths...
Clean up done...
Final clean up done...

Preprocessing completed for 119947 articles!


In [16]:
# df_aus['word_count'] = df_aus.preprocessed_sentence.apply(lambda x: len(x))
df_aus['preprocessed_sentence'].iloc[0]

['forming',
 'company',
 'family',
 'much',
 'land',
 'hand',
 'farming',
 'children',
 'part',
 'succession',
 'planning',
 'final',
 'decision',
 'inevitably',
 'different',
 'every',
 'family',
 'work',
 'issues',
 'number',
 'children',
 'number',
 'farming',
 'children',
 'amount',
 'farm',
 'debt',
 'provisions',
 'made',
 'parents',
 'retirement',
 'couple',
 'farming',
 'southern',
 'succession',
 'plan',
 'give',
 'farming',
 'sons',
 'running',
 'property',
 'still',
 'retaining',
 'control',
 'land',
 'started',
 'thinking',
 'planning',
 'early',
 'prompted',
 'mainly',
 'accountant',
 'suggested',
 'could',
 'form',
 'company',
 'reduce',
 'taxation',
 'payments',
 'would',
 'turn',
 'give',
 'ability',
 'funds',
 'farm',
 'superannuation',
 'seeking',
 'professional',
 'advice',
 'family',
 'company',
 'husband',
 'wife',
 'directors',
 'family',
 'trust',
 'also',
 'land',
 'bought',
 'either',
 'company',
 'trust',
 'name',
 'required',
 'working',
 'presumption',
 'lea

In [17]:
df_aus.head()

,text,region_of_origin,publication_date,publisher_name,preprocessed_sentence
3790,"Forming a company key for one family HOW, wh...",AUSNZ AUSTR,1120089600000,West Australian Newspapers Limited,"[forming, company, family, much, land, hand, f..."
4390,"There are many different types of homebuyers, ...",AUSNZ AUSTR,1493424000000,Fairfax Media Management Pty Limited,"[many, different, types, homebuyers, understan..."
4391,Off-the-plan contracts review The NSW Governme...,AUSNZ AUSTR,1517616000000,Fairfax Media Management Pty Limited,"[plan, contracts, review, government, released..."
4392,'Super must rise to 12%': MP Whitlam MP Stephe...,AUSNZ AUSTR,1570579200000,Fairfax Media Management Pty Limited,"[super, must, rise, whitlam, stephen, jones, s..."
4393,Where there's a will TODAY I'd like to talk ab...,AUSNZ AUSTR,1558742400000,Fairfax Media Management Pty Limited,"[today, like, talk, death, attention, heaven, ..."


### Adding Bigrams and Trigrams

In [18]:
# Bigrams
bigrams_phrases = gensim.models.Phrases(df_aus.preprocessed_sentence.tolist(), min_count=4, threshold=50)
print('Bigram phrases done...')

# Trigrams
trigrams_phrases = gensim.models.Phrases(bigrams_phrases[df_aus.preprocessed_sentence.tolist()], threshold=50)
print('Trigram phrases done...')

bigram = gensim.models.phrases.Phraser(bigrams_phrases)
trigram = gensim.models.phrases.Phraser(trigrams_phrases)
print('Bigram & Trigram done...')

def make_bigrams(texts):
    return(bigram[doc] for doc in texts)

def make_trigrams(texts):
    return(trigram[bigram[doc]] for doc in texts)

data_bigrams = make_bigrams(df_aus.preprocessed_sentence.tolist())
data_bigrams_trigrams = make_trigrams(data_bigrams)



Bigram phrases done...
Trigram phrases done...
Bigram & Trigram done...


In [19]:
# data_bigrams_trigrams_list = list(data_bigrams_trigrams)

In [20]:
df_aus['preprocessed_list_with_bi_tri'] = list(data_bigrams_trigrams)
df_aus['preprocessed_len'] = df_aus.preprocessed_list_with_bi_tri.apply(lambda x: len(x))
df_aus = df_aus[df_aus.preprocessed_len > 0]
df_aus.shape
# Stemming each word in each row
df_aus['preprocessed_list_with_bi_tri'] = df_aus.preprocessed_list_with_bi_tri.apply(lambda x: np.vectorize(snowball_stemm)(np.array(x)))

In [21]:
from gensim.models import TfidfModel

id2word = corpora.Dictionary(df_aus.preprocessed_list_with_bi_tri.tolist())

corpus = [id2word.doc2bow(text) for text in df_aus.preprocessed_list_with_bi_tri.tolist()]
# print(corpus[0][0:20])

tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.03
words = []
words_missing_in_tfidf = []

for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = [] #reinitialize to be safe. You can skip this.
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]

    #reassign
    corpus[i] = new_bow

In [22]:
print(f'Total number of articles: {len(corpus)}')

# Adding bigram and trigram list as a new column
# df_aus['preprocessed_list_with_bi_tri'] = data_bigrams_trigrams_list

# Calculating the words in each row and removing lists with no words
# df_aus['preprocessed_len'] = df_aus.tfidf_sentence.apply(lambda x: len(x))
# df_aus = df_aus[df_aus.preprocessed_len > 0]
# df_aus.shape

Total number of articles: 119947


In [23]:
df_aus.head()

,text,region_of_origin,publication_date,publisher_name,preprocessed_sentence,preprocessed_list_with_bi_tri,preprocessed_len
3790,"Forming a company key for one family HOW, wh...",AUSNZ AUSTR,1120089600000,West Australian Newspapers Limited,"[forming, company, family, much, land, hand, f...","[form, compani, famili, much, land, hand, farm...",320
4390,"There are many different types of homebuyers, ...",AUSNZ AUSTR,1493424000000,Fairfax Media Management Pty Limited,"[many, different, types, homebuyers, understan...","[mani, differ, type, homebuy, understand, help...",68
4391,Off-the-plan contracts review The NSW Governme...,AUSNZ AUSTR,1517616000000,Fairfax Media Management Pty Limited,"[plan, contracts, review, government, released...","[plan, contract, review, govern, releas, discu...",59
4392,'Super must rise to 12%': MP Whitlam MP Stephe...,AUSNZ AUSTR,1570579200000,Fairfax Media Management Pty Limited,"[super, must, rise, whitlam, stephen, jones, s...","[super, must, rise, whitlam, stephen_jon, say,...",130
4393,Where there's a will TODAY I'd like to talk ab...,AUSNZ AUSTR,1558742400000,Fairfax Media Management Pty Limited,"[today, like, talk, death, attention, heaven, ...","[today, like, talk, death, attent, heaven_sak,...",333


In [24]:
# dictionary = corpora.Dictionary(df_aus['preprocessed_list_with_bi_tri'].tolist())

# bow_corpus = [dictionary.doc2bow(text) for text in df_aus['preprocessed_list_with_bi_tri'].tolist()]

In [36]:
# Save the dictionary to a file

save_dict_bow(id2word, corpus,'AUS-tfidf')

KeyboardInterrupt: 

In [9]:
ldamodel = gensim.models.ldamulticore.LdaMulticore(bow_corpus, num_topics=60, id2word = dictionary, passes=40, workers=15,random_state=42)

In [10]:
# save the model
ldamodel.save("lda_model_with_bigrams_and_trigrams-cc-aus-tfidf.model")

In [7]:
import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, bow_corpus, dictionary)
vis

c:\Users\E116189\RMIT\Topic-modelling\framework\tpm\lib\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
c:\Users\E116189\RMIT\Topic-modelling\framework\tpm\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
30    -0.220318 -0.051544       1        1  5.826207
17     0.261823 -0.025350       2        1  4.603586
3     -0.025096  0.025040       3        1  4.107518
1     -0.130163 -0.009181       4        1  3.340173
4      0.319855 -0.067893       5        1  3.256991
40    -0.136465 -0.227689       6        1  3.127427
28    -0.159084 -0.092973       7        1  3.076452
29    -0.120864 -0.193246       8        1  3.065801
32     0.065505 -0.143645       9        1  2.980601
42    -0.165289  0.150662      10        1  2.782548
43    -0.060075 -0.117918      11        1  2.779824
24    -0.127723 -0.142867      12        1  2.429991
33     0.133177 -0.066628      13        1  2.400020
37     0.120318  0.046122      14        1  2.388910
12    -0.040670 -0.046071      15        1  2.067097
39    -0.068827 -0.066589      16        1  2.015004
19    -0.068842  0.085164      17        1  1.973572
54     0.228437 -0.042352      18        1  1.893895
52    -0.104012 -0.230522      19        1  1.811820
25     0.110118 -0.078048      20        1  1.796513
50    -0.087972 -0.171641      21        1  1.648511
59    -0.139890  0.157332      22        1  1.618350
34    -0.085845 -0.026642      23        1  1.572916
53     0.090488  0.115563      24        1  1.544281
7     -0.048650 -0.055529      25        1  1.513464
35    -0.085222  0.129780      26        1  1.469411
15     0.258209  0.041166      27        1  1.452596
14    -0.046655  0.230814      28        1  1.404502
22    -0.132079  0.114150      29        1  1.358812
56    -0.151081 -0.010433      30        1  1.339171
11     0.116320 -0.075585      31        1  1.287692
0     -0.102610 -0.087652      32        1  1.278888
23    -0.127018 -0.088687      33        1  1.262374
44     0.071024 -0.189271      34        1  1.204866
8     -0.049436  0.047165      35        1  1.176241
18    -0.092326  0.228331      36        1  1.172890
5     -0.070407  0.054829      37        1  1.166543
57    -0.043533  0.097942      38        1  1.159478
27    -0.084897 -0.174558      39        1  1.149141
49    -0.095778  0.152266      40        1  1.064213
58     0.284981 -0.092311      41        1  0.991562
36    -0.093379  0.042409      42        1  0.980183
45    -0.118875 -0.002344      43        1  0.979571
2     -0.008140  0.138408      44        1  0.972630
13     0.066331 -0.023539      45        1  0.972440
10     0.126209  0.013479      46        1  0.951606
21    -0.054778 -0.054372      47        1  0.928840
31     0.237402  0.093690      48        1  0.883466
41    -0.021920  0.169355      49        1  0.876700
55     0.231312 -0.023454      50        1  0.860602
47     0.040461  0.133293      51        1  0.847129
51     0.024560 -0.132785      52        1  0.823215
20    -0.122971  0.109049      53        1  0.823170
16     0.044311  0.015380      54        1  0.746316
6     -0.061870  0.011514      55        1  0.708472
38     0.060216  0.183422      56        1  0.561168
9      0.151102  0.029652      57        1  0.526869
48     0.091875  0.073415      58        1  0.472368
26     0.150464 -0.021139      59        1  0.290274
46     0.068265  0.143065      60        1  0.235127, topic_info=         Term           Freq          Total Category  logprob  loglift
87     invest  359889.000000  359889.000000  Default  30.0000  30.0000
302       say  188094.000000  188094.000000  Default  29.0000  29.0000
194      year  170844.000000  170844.000000  Default  28.0000  28.0000
71       fund  373725.000000  373725.000000  Default  27.0000  27.0000
156      said  176091.000000  176091.000000  Default  26.0000  26.0000
...       ...            ...            ...      ...      ...      ...
9616    grade     321.160492    1469.230316  Topic60  -5.3387   4.5322
1048   dollar     480.618639   20338.248013  Topic60  -4.9356   2.3076
214     price     385.388349  10

In [8]:
pyLDAvis.save_html(vis, 'lda_model_with_bigrams_and_trigrams_aus-tfidf.html')


# Loading and rerunning

In [27]:
dictionary, bow_corpus = load_dict_bow('AUS-tfidf')

In [28]:
from gensim import  models
lda_model = models.ldamodel.LdaModel.load("../../../australia/lda_model_with_bigrams_and_trigrams-cc-aus-tfidf.model")
# lda_model_id2w = models.ldamodel.LdaModel.load("lda_model.model.id2word")

In [30]:
import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, bow_corpus, dictionary)
vis

c:\Users\E116189\RMIT\Topic-modelling\framework\tpm\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
30    -0.197096 -0.079445       1        1  5.853563
17     0.262776  0.004105       2        1  4.989399
3     -0.026940 -0.001443       3        1  4.401122
4      0.324164 -0.057749       4        1  3.453437
29    -0.072416 -0.217005       5        1  3.372830
40    -0.111562 -0.211586       6        1  3.331708
28    -0.153486 -0.108991       7        1  3.292698
1     -0.131320 -0.021686       8        1  3.195336
32     0.080665 -0.163762       9        1  2.723787
43    -0.041493 -0.111763      10        1  2.722061
33     0.131499 -0.104813      11        1  2.691875
24    -0.130155 -0.139728      12        1  2.357171
37     0.102959  0.048626      13        1  2.324942
19    -0.083336  0.031232      14        1  2.116112
42    -0.146635  0.167176      15        1  1.942631
12    -0.040168 -0.020928      16        1  1.910080
39    -0.100351 -0.098802      17        1  1.882858
54     0.251645  0.011048      18        1  1.831439
7     -0.069136 -0.085804      19        1  1.720033
59    -0.169627  0.130555      20        1  1.717684
50    -0.088914 -0.141907      21        1  1.715512
52    -0.065598 -0.197288      22        1  1.667229
22    -0.164591  0.117851      23        1  1.634154
53     0.098730  0.163520      24        1  1.577403
25     0.130731 -0.140648      25        1  1.575488
34    -0.103708 -0.022214      26        1  1.548899
14    -0.054019  0.228283      27        1  1.473139
35    -0.093472  0.106102      28        1  1.459518
56    -0.138392  0.000288      29        1  1.452315
57    -0.032263 -0.030566      30        1  1.444479
23    -0.120174 -0.116891      31        1  1.417946
15     0.229243  0.102203      32        1  1.350744
11     0.164614 -0.011707      33        1  1.330974
18    -0.144338  0.209588      34        1  1.320759
5     -0.092856  0.048480      35        1  1.294017
0     -0.100088 -0.049244      36        1  1.275609
44     0.086386 -0.203933      37        1  1.259288
8     -0.047752  0.072012      38        1  1.162991
27    -0.045367 -0.128011      39        1  1.052077
45    -0.129110 -0.021324      40        1  1.022564
2     -0.016219  0.142409      41        1  1.006868
58     0.304965 -0.068247      42        1  1.001632
13     0.077187 -0.117487      43        1  0.974871
36    -0.089892  0.061838      44        1  0.939938
21    -0.055796 -0.037038      45        1  0.938257
49    -0.089780  0.196872      46        1  0.916778
47     0.018788  0.154910      47        1  0.912674
41    -0.048941  0.170823      48        1  0.894491
16    -0.031068 -0.044182      49        1  0.870729
55     0.257967 -0.036006      50        1  0.798090
51     0.043527 -0.142407      51        1  0.782495
20    -0.158631  0.058678      52        1  0.732864
31     0.213869  0.119913      53        1  0.618955
6     -0.078550  0.028998      54        1  0.592776
38     0.044328  0.211966      55        1  0.579010
9      0.165463  0.065098      56        1  0.523747
48     0.150602  0.099687      57        1  0.364278
26     0.174247  0.039411      58        1  0.266033
10     0.073477 -0.022165      59        1  0.229167
46     0.075404  0.163100      60        1  0.190474, topic_info=         Term           Freq          Total Category  logprob  loglift
194      year  168159.000000  168159.000000  Default  30.0000  30.0000
156      said  175236.000000  175236.000000  Default  29.0000  29.0000
71       fund  372091.000000  372091.000000  Default  28.0000  28.0000
87     invest  357460.000000  357460.000000  Default  27.0000  27.0000
704      rate  170502.000000  170502.000000  Default  26.0000  26.0000
...       ...            ...            ...      ...      ...      ...
1622    class     242.452931   12841.862695  Topic60  -5.4093   2.2938
1048   dollar     242.224627   20186.812096  Topic60  -5.4102   1.8405
214     price     277.616265  10

In [31]:
lambd = 0.2 # a specific relevance metric value

all_topics = {}
num_topics = lda_model.num_topics
num_terms = 61

for i in range(1,num_topics+1): ## Correct range
    topic = vis.topic_info[vis.topic_info.Category == 'Topic'+str(i)].copy()
    topic['relevance'] = topic['loglift']*(1-lambd)+topic['logprob']*lambd

    all_topics['Topic '+str(i)] = topic.sort_values(by='relevance', ascending=False).Term[:num_terms].values
    all_topics['Topic '+str(i)] = all_topics['Topic '+str(i)] +' '+ round(topic['relevance'][:num_terms], 3).astype(str).values
# pd.DataFrame(all_topics)
all_topics = pd.DataFrame(all_topics)
all_topics

,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6,Topic 7,Topic 8,Topic 9,Topic 10,...,Topic 51,Topic 52,Topic 53,Topic 54,Topic 55,Topic 56,Topic 57,Topic 58,Topic 59,Topic 60
0,time 0.456,percent 0.975,economi 0.83,bankia 1.442,would 1.234,contribut 1.429,retir 1.117,labor 1.005,bond 1.52,member 1.271,...,annuiti 2.483,hous 2.242,canada 3.109,account 3.131,million 2.77,mini 3.762,elli 3.448,warrant 4.275,ethic 3.721,gold 4.011
1,thing 0.489,quarter 0.945,recess 0.79,outflow 1.332,answers_used_gener 1.205,salary_sacrific 1.091,asfa 1.073,govern 0.988,risk 1.388,fund 1.198,...,lifetime_annu 2.768,housing_afford 2.214,canadian 2.772,unclaim 2.787,packer 2.713,mini_short 3.463,rank_stocks_posit 3.352,underlying_ent 3.887,socially_respons 3.801,stamp 4.122
2,money 0.429,compar 0.91,bond_yield 0.783,inflow 1.22,sought_mak 1.188,surcharg 0.979,gerard_light 1.357,coalit 0.961,hybrid 1.268,mysup 1.022,...,stream 2.336,boomer 2.201,ontario 2.579,lost 2.785,pratt 2.841,stop_loss 3.414,aveo 3.522,self_funding_instal 3.816,robo_advic 3.751,ounc 3.948
3,know 0.357,revenu 0.884,inflat 0.799,liontrust 1.196,noel_whittaker_director 1.397,concessional_contribut 0.956,comfortable_lifestyl 0.928,morrison 0.944,cfds 1.207,choic 0.989,...,deferred_annu 2.279,sabra_lan 2.198,pharmaceut 2.54,churchil 2.709,boe 2.853,strike_pric 3.153,murdoch 3.233,expiry_d 3.74,robo_advis 3.481,margin_right_nbsp 3.898
4,think 0.345,decreas 0.875,commodity_pric 0.754,client 1.16,wife 1.141,salary_sacrif 0.946,save 0.832,reform 0.931,volatil 1.188,master_trust 0.98,...,cipr 2.246,ruddock 2.181,evest 2.517,lost_unclaim 2.7,talbot 2.579,strike_price_stop_loss 3.125,shares_race_fantasi 3.195,instalment_pay 3.576,stockspot 3.467,robinson 3.883
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,celebrity_chat 0.729,chiba_bank 0.8,chief_economist_bill_evan 0.833,institut 0.666,possibl 0.408,surcharge_abolish 0.947,standard 0.837,senator_coonan 1.117,reading_t 0.991,chronic_underperform 1.025,...,lump 0.75,young 1.576,manufactur 2.057,australian 0.804,russian 2.272,inform 1.347,close 0.307,exchang 0.695,autom 3.003,class 0.753
57,budget 0.733,concordia 0.828,sinc 0.947,lead 0.717,could 0.347,superannuation_offset_listo 0.942,myretir 0.813,argu 1.399,broker_fiig 0.872,andrew_keev 1.134,...,life 1.393,generat 1.645,billion 1.707,detail 0.918,aircraft 2.061,exampl 1.201,worth 1.225,juli 1.133,green 2.759,hedg 0.39
58,principal_adviser_castellan 0.63,banco_santander_xico 0.726,shaped_recoveri 0.892,equiti 0.45,advic 0.503,rule 0.925,gary_savag 0.71,spokesman_nick_sherri 1.25,share 0.962,proebstl 1.154,...,regular 0.993,demograph 1.382,china 1.811,offic 0.987,walker 1.858,issu 1.74,second 1.512,call 1.379,sustain 2.548,dollar -0.789
59,canstar_bruce_brammal 0.596,scotiabank 0.765,rise 0.838,privat 0.293,howev 0.501,transitional_measur 0.922,podger 0.691,guarante 1.101,appendix_page_appendix 0.952,ross_cameron 1.034,...,offer 0.576,popul 1.403,healthcar 1.691,keep 1.097,empir 2.245,cost 1.893,sector 1.557,payment 1.628,environment 0.844,billion 0.448


In [32]:
all_topics.to_csv("top-61-words-australia.csv", index=False)

In [33]:
from tqdm import tqdm
from functools import reduce
from collections import Counter

In [34]:
df_aus.preprocessed_sentence.iloc[0]

['forming',
 'company',
 'family',
 'much',
 'land',
 'hand',
 'farming',
 'children',
 'part',
 'succession',
 'planning',
 'final',
 'decision',
 'inevitably',
 'different',
 'every',
 'family',
 'work',
 'issues',
 'number',
 'children',
 'number',
 'farming',
 'children',
 'amount',
 'farm',
 'debt',
 'provisions',
 'made',
 'parents',
 'retirement',
 'couple',
 'farming',
 'southern',
 'succession',
 'plan',
 'give',
 'farming',
 'sons',
 'running',
 'property',
 'still',
 'retaining',
 'control',
 'land',
 'started',
 'thinking',
 'planning',
 'early',
 'prompted',
 'mainly',
 'accountant',
 'suggested',
 'could',
 'form',
 'company',
 'reduce',
 'taxation',
 'payments',
 'would',
 'turn',
 'give',
 'ability',
 'funds',
 'farm',
 'superannuation',
 'seeking',
 'professional',
 'advice',
 'family',
 'company',
 'husband',
 'wife',
 'directors',
 'family',
 'trust',
 'also',
 'land',
 'bought',
 'either',
 'company',
 'trust',
 'name',
 'required',
 'working',
 'presumption',
 'lea

In [35]:
# Assume `new_doc` is a list of tokens representing a new document
new_topics = []
for i in tqdm(range(0,df_aus.shape[0])):

    word_count = dict(Counter(df_aus.preprocessed_list_with_bi_tri.iloc[i]))

    top_words = dict((k, v) for k, v in word_count.items() if v >= 3)

    # print(topic_words)
    new_doc_topics = []
    for j in  range(1, num_topics+1):
        topic_words = all_topics['Topic ' + str(j)]
        word_list = topic_words.apply(lambda x: x.split(' ')[0])

        # Find the intersection of words between the topic words and the word list
        intersection = set(list(top_words.keys())).intersection(word_list)
        # print(j,intersection)

        # Get a list of relevance scores for each intersected word
        relevance = [float(topic_words[topic_words.str.contains(i)].str.split().str[-1].values[0]) for i in intersection if len(i) > 0]
        # print(relevance)

        # Check if there are more than 1 relevance scores
        if len(relevance) > 0:
            result = reduce(lambda x, y: x*y, relevance)
        else:
            result = 0
        new_doc_topics.append((j, result))
    # print(new_doc_topics)
    new_doc_topics.sort(key=lambda x: x[1], reverse=True)
    # print(new_doc_topics)
    all_zeros = all(tup[1] == 0 for tup in new_doc_topics)

    if all_zeros:
        new_topics.append(1)
    else:
        new_topics.append(new_doc_topics[0][0])

100%|██████████| 119947/119947 [36:16<00:00, 55.12it/s] 


In [36]:
# Assign the predicted topics to the articles
df_aus['topics'] = new_topics

# Append the topic numbers with the word 'topic'
df_aus['topics'] = df_aus.topics.apply(lambda x: f'topic {str(x)}')

# Check the distribution of topics
print(df_aus.topics.value_counts())

# Check if there are any inappropriate column names
print(df_aus.columns)

topic 25    11523
topic 56     9695
topic 51     7326
topic 40     7072
topic 57     5776
topic 26     4896
topic 54     4756
topic 21     3837
topic 31     3523
topic 50     3223
topic 1      3033
topic 32     2955
topic 45     2923
topic 16     2816
topic 43     2522
topic 35     2444
topic 24     2408
topic 44     2386
topic 47     2365
topic 52     2298
topic 49     2227
topic 14     1697
topic 33     1680
topic 29     1646
topic 27     1586
topic 20     1533
topic 60     1477
topic 38     1390
topic 55     1340
topic 37     1322
topic 22     1294
topic 19     1279
topic 58     1190
topic 36     1175
topic 39     1134
topic 53      993
topic 41      980
topic 42      974
topic 28      781
topic 30      768
topic 59      733
topic 23      697
topic 12      651
topic 46      640
topic 18      564
topic 48      503
topic 34      450
topic 17      441
topic 9       238
topic 7       151
topic 4       110
topic 10      104
topic 11       91
topic 6        74
topic 8        73
topic 13  

In [37]:
original_topics = {'Topic 25': ['fee', 'charg', 'credit_union', 'cannex', 'bankwest'],\
                    'Topic 51': ['busi', 'crypto', 'franchis', 'cryptocurr', 'avion', 'ventur', 'small'],\
                    'Topic 57': ['mini', 'mini_short', 'stop_loss', 'strike_pric', 'strike_price_stop_loss'],\
                    'Topic 32': ['aged_car'],\
                    'Topic 45': ['properti', 'residenti', 'real_est','smsf'],\
                    'Topic 16': ['rate', 'variabl', 'interest'],\
                    'Topic 43': ['first', 'homebuy'],
                    'Topic 35': ['card', 'scam', 'phone', 'debit_card'],
                    'Topic 47': ['insur', 'cover', 'premium'],\
                    'Topic 52': ['annuiti', 'lifetime_annu', 'stream', 'incom', 'term', 'product'],\
                    'Topic 49': ['car', 'petrol', 'travel', 'automot', 'ford'],\
                    'Topic 29': ['women', 'work', 'gender'],\
                    'Topic 37': ['health', 'hospit','privat'],
                    'Topic 19': ['pension', 'allocated_pens'],\
                    'Topic 53': ['hous', 'millenni'],
                    'Topic 23': ['lender', 'broker', 'bank'],
                    'Topic 46': [ 'robo_advis'],
                    'Topic 18': ['director', 'board', 'remuner'],
                    'Topic 21': ['test', 'pension']}
corrected_topics = {}
for i in range(1,61):
    # if ['fee', 'charg', 'credit_union'] in all_topics[f'Topic {i}'].apply(lambda x: x.split(' ')[0]).values:
    if f'Topic {i}' in list(original_topics.keys()):
        for j in range(1,61):
            if all(word in all_topics[f'Topic {j}'].apply(lambda x: x.split(' ')[0]).values for word in original_topics[f'Topic {i}']):
                print(f'Topic {i}: {j}')
                corrected_topics.update({f'Topic {i}': f'topic {j}'})

Topic 16: 17
Topic 18: 18
Topic 19: 22
Topic 21: 21
Topic 23: 26
Topic 25: 19
Topic 29: 23
Topic 32: 36
Topic 35: 38
Topic 37: 35
Topic 43: 40
Topic 45: 43
Topic 46: 59
Topic 47: 45
Topic 49: 48
Topic 51: 47
Topic 52: 51
Topic 53: 52
Topic 57: 56


In [38]:
corrected_topics

{'Topic 16': 'topic 17',
 'Topic 18': 'topic 18',
 'Topic 19': 'topic 22',
 'Topic 21': 'topic 21',
 'Topic 23': 'topic 26',
 'Topic 25': 'topic 19',
 'Topic 29': 'topic 23',
 'Topic 32': 'topic 36',
 'Topic 35': 'topic 38',
 'Topic 37': 'topic 35',
 'Topic 43': 'topic 40',
 'Topic 45': 'topic 43',
 'Topic 46': 'topic 59',
 'Topic 47': 'topic 45',
 'Topic 49': 'topic 48',
 'Topic 51': 'topic 47',
 'Topic 52': 'topic 51',
 'Topic 53': 'topic 52',
 'Topic 57': 'topic 56'}

In [39]:
df_aus

,text,region_of_origin,publication_date,publisher_name,preprocessed_sentence,preprocessed_list_with_bi_tri,preprocessed_len,topics
3790,"Forming a company key for one family HOW, wh...",AUSNZ AUSTR,1120089600000,West Australian Newspapers Limited,"[forming, company, family, much, land, hand, f...","[form, compani, famili, much, land, hand, farm...",320,topic 20
4390,"There are many different types of homebuyers, ...",AUSNZ AUSTR,1493424000000,Fairfax Media Management Pty Limited,"[many, different, types, homebuyers, understan...","[mani, differ, type, homebuy, understand, help...",68,topic 40
4391,Off-the-plan contracts review The NSW Governme...,AUSNZ AUSTR,1517616000000,Fairfax Media Management Pty Limited,"[plan, contracts, review, government, released...","[plan, contract, review, govern, releas, discu...",59,topic 36
4392,'Super must rise to 12%': MP Whitlam MP Stephe...,AUSNZ AUSTR,1570579200000,Fairfax Media Management Pty Limited,"[super, must, rise, whitlam, stephen, jones, s...","[super, must, rise, whitlam, stephen_jon, say,...",130,topic 28
4393,Where there's a will TODAY I'd like to talk ab...,AUSNZ AUSTR,1558742400000,Fairfax Media Management Pty Limited,"[today, like, talk, death, attention, heaven, ...","[today, like, talk, death, attent, heaven_sak,...",333,topic 46
...,...,...,...,...,...,...,...,...
1508690,Super choice brings costs ONE in two small b...,AUSNZ AUSTR,1130889600000,Nationwide News Pty Ltd.,"[super, choice, brings, costs, small, business...","[super, choic, bring, cost, small, busi, owner...",42,topic 47
1508691,"Home loans guide NO doc, low doc, combinatio...",AUSNZ AUSTR,1155686400000,Nationwide News Pty Ltd.,"[home, loans, guide, combination, split, honey...","[home, loan, guid, combin, split, honeymoon, s...",82,topic 50
1508692,Taxing times for families TAXPAYERS having the...,AUSNZ AUSTR,1216771200000,Nationwide News Pty. Ltd.,"[taxing, times, families, taxpayers, returns, ...","[tax, time, famili, taxpay, return, prepar, aw...",112,topic 36
1508697,Legal papers vital Legal papers vital W...,AUSNZ AUSTR,1188950400000,Nationwide News Pty Ltd.,"[legal, papers, vital, legal, papers, vital, c...","[legal, paper, vital, legal, paper, vital, cen...",158,topic 43


In [40]:
# df_aus.head()
df_aus['topics'] = df_aus.topics.apply(lambda x: x[0].upper()+x[1:])
df_aus['corrected_topics'] = df_aus.topics.map(corrected_topics).fillna(df_aus.topics)

In [41]:
df_aus['corrected_topics'] = df_aus.corrected_topics.apply(lambda x: x.lower())

In [42]:
df_aus

,text,region_of_origin,publication_date,publisher_name,preprocessed_sentence,preprocessed_list_with_bi_tri,preprocessed_len,topics,corrected_topics
3790,"Forming a company key for one family HOW, wh...",AUSNZ AUSTR,1120089600000,West Australian Newspapers Limited,"[forming, company, family, much, land, hand, f...","[form, compani, famili, much, land, hand, farm...",320,Topic 20,topic 20
4390,"There are many different types of homebuyers, ...",AUSNZ AUSTR,1493424000000,Fairfax Media Management Pty Limited,"[many, different, types, homebuyers, understan...","[mani, differ, type, homebuy, understand, help...",68,Topic 40,topic 40
4391,Off-the-plan contracts review The NSW Governme...,AUSNZ AUSTR,1517616000000,Fairfax Media Management Pty Limited,"[plan, contracts, review, government, released...","[plan, contract, review, govern, releas, discu...",59,Topic 36,topic 36
4392,'Super must rise to 12%': MP Whitlam MP Stephe...,AUSNZ AUSTR,1570579200000,Fairfax Media Management Pty Limited,"[super, must, rise, whitlam, stephen, jones, s...","[super, must, rise, whitlam, stephen_jon, say,...",130,Topic 28,topic 28
4393,Where there's a will TODAY I'd like to talk ab...,AUSNZ AUSTR,1558742400000,Fairfax Media Management Pty Limited,"[today, like, talk, death, attention, heaven, ...","[today, like, talk, death, attent, heaven_sak,...",333,Topic 46,topic 59
...,...,...,...,...,...,...,...,...,...
1508690,Super choice brings costs ONE in two small b...,AUSNZ AUSTR,1130889600000,Nationwide News Pty Ltd.,"[super, choice, brings, costs, small, business...","[super, choic, bring, cost, small, busi, owner...",42,Topic 47,topic 45
1508691,"Home loans guide NO doc, low doc, combinatio...",AUSNZ AUSTR,1155686400000,Nationwide News Pty Ltd.,"[home, loans, guide, combination, split, honey...","[home, loan, guid, combin, split, honeymoon, s...",82,Topic 50,topic 50
1508692,Taxing times for families TAXPAYERS having the...,AUSNZ AUSTR,1216771200000,Nationwide News Pty. Ltd.,"[taxing, times, families, taxpayers, returns, ...","[tax, time, famili, taxpay, return, prepar, aw...",112,Topic 36,topic 36
1508697,Legal papers vital Legal papers vital W...,AUSNZ AUSTR,1188950400000,Nationwide News Pty Ltd.,"[legal, papers, vital, legal, papers, vital, c...","[legal, paper, vital, legal, paper, vital, cen...",158,Topic 43,topic 40


In [43]:
df_aus[~df_aus.corrected_topics.isna()].to_csv('fwb-australia-topic-prediction.csv', index=False)

In [44]:
# Convert the timestamps to a proper datetime format
df_aus['publication_datetime'] = pd.to_datetime(df_aus['publication_date'], unit='ms')

# Retain just the date and drop the time
df_aus['publication_date'] = df_aus.publication_datetime.dt.date
df_aus

,text,region_of_origin,publication_date,publisher_name,preprocessed_sentence,preprocessed_list_with_bi_tri,preprocessed_len,topics,corrected_topics,publication_datetime
3790,"Forming a company key for one family HOW, wh...",AUSNZ AUSTR,2005-06-30,West Australian Newspapers Limited,"[forming, company, family, much, land, hand, f...","[form, compani, famili, much, land, hand, farm...",320,Topic 20,topic 20,2005-06-30
4390,"There are many different types of homebuyers, ...",AUSNZ AUSTR,2017-04-29,Fairfax Media Management Pty Limited,"[many, different, types, homebuyers, understan...","[mani, differ, type, homebuy, understand, help...",68,Topic 40,topic 40,2017-04-29
4391,Off-the-plan contracts review The NSW Governme...,AUSNZ AUSTR,2018-02-03,Fairfax Media Management Pty Limited,"[plan, contracts, review, government, released...","[plan, contract, review, govern, releas, discu...",59,Topic 36,topic 36,2018-02-03
4392,'Super must rise to 12%': MP Whitlam MP Stephe...,AUSNZ AUSTR,2019-10-09,Fairfax Media Management Pty Limited,"[super, must, rise, whitlam, stephen, jones, s...","[super, must, rise, whitlam, stephen_jon, say,...",130,Topic 28,topic 28,2019-10-09
4393,Where there's a will TODAY I'd like to talk ab...,AUSNZ AUSTR,2019-05-25,Fairfax Media Management Pty Limited,"[today, like, talk, death, attention, heaven, ...","[today, like, talk, death, attent, heaven_sak,...",333,Topic 46,topic 59,2019-05-25
...,...,...,...,...,...,...,...,...,...,...
1508690,Super choice brings costs ONE in two small b...,AUSNZ AUSTR,2005-11-02,Nationwide News Pty Ltd.,"[super, choice, brings, costs, small, business...","[super, choic, bring, cost, small, busi, owner...",42,Topic 47,topic 45,2005-11-02
1508691,"Home loans guide NO doc, low doc, combinatio...",AUSNZ AUSTR,2006-08-16,Nationwide News Pty Ltd.,"[home, loans, guide, combination, split, honey...","[home, loan, guid, combin, split, honeymoon, s...",82,Topic 50,topic 50,2006-08-16
1508692,Taxing times for families TAXPAYERS having the...,AUSNZ AUSTR,2008-07-23,Nationwide News Pty. Ltd.,"[taxing, times, families, taxpayers, returns, ...","[tax, time, famili, taxpay, return, prepar, aw...",112,Topic 36,topic 36,2008-07-23
1508697,Legal papers vital Legal papers vital W...,AUSNZ AUSTR,2007-09-05,Nationwide News Pty Ltd.,"[legal, papers, vital, legal, papers, vital, c...","[legal, paper, vital, legal, paper, vital, cen...",158,Topic 43,topic 40,2007-09-05


In [45]:
# Split the year and months
df_aus['publication_year'] = df_aus.publication_datetime.dt.year
df_aus['publication_month'] = df_aus.publication_datetime.dt.month

# prefix a 0 to the months
df_aus['publication_month'] = df_aus.publication_month.apply(lambda x: '0'+str(x) if len(str(x)) < 2 else str(x))

In [46]:
df_grouped_df = df_aus.groupby(['corrected_topics', 'publication_year', 'publication_month']).count()[['text']].reset_index()

In [47]:
df_grouped_df

,corrected_topics,publication_year,publication_month,text
0,topic 1,1991,12,1
1,topic 1,1995,11,1
2,topic 1,1996,02,1
3,topic 1,1996,03,1
4,topic 1,1996,11,2
...,...,...,...,...
11584,topic 9,2022,04,2
11585,topic 9,2022,05,1
11586,topic 9,2022,07,2
11587,topic 9,2022,09,3


In [49]:
df_grouped_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11589 entries, 0 to 11588
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   corrected_topics   11589 non-null  object
 1   publication_year   11589 non-null  int64 
 2   publication_month  11589 non-null  object
 3   text               11589 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 362.3+ KB


In [51]:
topic_names_inc_exc = pd.read_excel('../../../australia/Financial_wellbeing_topic_names.xlsx', sheet_name='in')

In [52]:
topic_names_inc_exc

,Topic Number,Article Count,Topic Name (created by research team),Included in Line Chart
0,25,11527,Banking,1.0
1,56,9697,Billionaires,NaN
2,51,7326,Start ups,1.0
3,40,7072,Criminal law,NaN
4,57,5779,Stock trading,1.0
5,26,4896,Pubilc statements (see 38),NaN
6,54,4759,Food and nutrition,NaN
7,21,3834,Social security and government benefits,1.0
8,31,3520,Trusts and lending,NaN
9,50,3230,Investment trusts,NaN


In [53]:
# Retain the appropriate topics only
imp_topics_df = topic_names_inc_exc[topic_names_inc_exc['Included in Line Chart'] == 1]

In [54]:
imp_topics_df

,Topic Number,Article Count,Topic Name (created by research team),Included in Line Chart
0,25,11527,Banking,1.0
2,51,7326,Start ups,1.0
4,57,5779,Stock trading,1.0
7,21,3834,Social security and government benefits,1.0
11,32,2955,Aged Care,1.0
12,45,2926,Property market,1.0
13,16,2816,Mortgages,1.0
14,43,2522,First homebuyers,1.0
15,35,2443,Financial scams,1.0
18,47,2366,Life insurance,1.0


In [55]:
imp_topics_df['Topic Number'] = imp_topics_df['Topic Number'].apply(lambda x: 'topic '+str(x))

c:\Users\E116189\RMIT\Topic-modelling\framework\tpm\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [56]:
imp_topics_df

,Topic Number,Article Count,Topic Name (created by research team),Included in Line Chart
0,topic 25,11527,Banking,1.0
2,topic 51,7326,Start ups,1.0
4,topic 57,5779,Stock trading,1.0
7,topic 21,3834,Social security and government benefits,1.0
11,topic 32,2955,Aged Care,1.0
12,topic 45,2926,Property market,1.0
13,topic 16,2816,Mortgages,1.0
14,topic 43,2522,First homebuyers,1.0
15,topic 35,2443,Financial scams,1.0
18,topic 47,2366,Life insurance,1.0


In [57]:
topics_renamed = dict(zip(imp_topics_df['Topic Number'].values, imp_topics_df['Topic Name (created by research team)'].values))

In [58]:
topics_renamed

{'topic 25': 'Banking',
 'topic 51': 'Start ups',
 'topic 57': 'Stock trading',
 'topic 21': 'Social security and government benefits',
 'topic 32': 'Aged Care',
 'topic 45': 'Property market',
 'topic 16': 'Mortgages',
 'topic 43': 'First homebuyers',
 'topic 35': 'Financial scams',
 'topic 47': 'Life insurance',
 'topic 52': 'Retirement',
 'topic 49': 'Transport',
 'topic 29': 'Workplace gender equality',
 'topic 37': 'Health care',
 'topic 19': 'Superannuation',
 'topic 53': 'Generational change in housing affordabiliy',
 'topic 28': 'Art markets',
 'topic 23': 'Banking royal commission',
 'topic 46': 'Ethical investing',
 'topic 18': 'Executive remuneration'}

In [59]:
df_grouped_df

,corrected_topics,publication_year,publication_month,text
0,topic 1,1991,12,1
1,topic 1,1995,11,1
2,topic 1,1996,02,1
3,topic 1,1996,03,1
4,topic 1,1996,11,2
...,...,...,...,...
11584,topic 9,2022,04,2
11585,topic 9,2022,05,1
11586,topic 9,2022,07,2
11587,topic 9,2022,09,3


In [60]:
df_grouped_df['renamed_topics'] = df_grouped_df.corrected_topics.map(topics_renamed)

In [61]:
df_grouped_df = df_grouped_df.dropna()
df_grouped_df

,corrected_topics,publication_year,publication_month,text,renamed_topics
1236,topic 18,1995,09,1,Executive remuneration
1237,topic 18,1996,05,1,Executive remuneration
1238,topic 18,1996,11,1,Executive remuneration
1239,topic 18,1997,04,1,Executive remuneration
1240,topic 18,1997,11,2,Executive remuneration
...,...,...,...,...,...
9480,topic 52,2022,06,6,Retirement
9481,topic 52,2022,07,9,Retirement
9482,topic 52,2022,08,6,Retirement
9483,topic 52,2022,09,1,Retirement


In [62]:
# df_grouped_df = df_grouped_df[df_grouped_df.publication_year >= 2010]
df_grouped_df

,corrected_topics,publication_year,publication_month,text,renamed_topics
1236,topic 18,1995,09,1,Executive remuneration
1237,topic 18,1996,05,1,Executive remuneration
1238,topic 18,1996,11,1,Executive remuneration
1239,topic 18,1997,04,1,Executive remuneration
1240,topic 18,1997,11,2,Executive remuneration
...,...,...,...,...,...
9480,topic 52,2022,06,6,Retirement
9481,topic 52,2022,07,9,Retirement
9482,topic 52,2022,08,6,Retirement
9483,topic 52,2022,09,1,Retirement


In [63]:
df_grouped_df.renamed_topics.unique()

array(['Executive remuneration', 'Superannuation',
       'Social security and government benefits',
       'Banking royal commission', 'Art markets', 'Financial scams',
       'First homebuyers', 'Property market', 'Life insurance',
       'Start ups', 'Retirement'], dtype=object)

In [64]:
for i in df_grouped_df.renamed_topics.unique():
    print(f'if (name === "{i}")')
    sub_df = df_grouped_df[df_grouped_df.renamed_topics == i]
    # print(sub_df.publication_year.astype(str) + '-' + sub_df.publication_month)
    result = dict(zip(sub_df.publication_year.astype(str) + '-' + sub_df.publication_month, sub_df.text))
    result = [{'year': key, f'value{i}': value} for key, value in result.items()]

    print(f'data = {result}; \n')

if (name === "Executive remuneration")
data = [{'year': '1995-09', 'valueExecutive remuneration': 1}, {'year': '1996-05', 'valueExecutive remuneration': 1}, {'year': '1996-11', 'valueExecutive remuneration': 1}, {'year': '1997-04', 'valueExecutive remuneration': 1}, {'year': '1997-11', 'valueExecutive remuneration': 2}, {'year': '1998-12', 'valueExecutive remuneration': 1}, {'year': '1999-02', 'valueExecutive remuneration': 2}, {'year': '1999-06', 'valueExecutive remuneration': 1}, {'year': '1999-09', 'valueExecutive remuneration': 1}, {'year': '1999-10', 'valueExecutive remuneration': 1}, {'year': '1999-11', 'valueExecutive remuneration': 1}, {'year': '2000-05', 'valueExecutive remuneration': 1}, {'year': '2000-06', 'valueExecutive remuneration': 1}, {'year': '2001-01', 'valueExecutive remuneration': 1}, {'year': '2001-02', 'valueExecutive remuneration': 1}, {'year': '2001-03', 'valueExecutive remuneration': 2}, {'year': '2001-06', 'valueExecutive remuneration': 2}, {'year': '2001-08'